In [1]:
import os
import torch
import openai

from transformers import GPT2Tokenizer
from transformers.generation_beam_search import BeamSearchScorer
from transformers.generation_logits_process import (
    LogitsProcessorList,
    HammingDiversityLogitsProcessor,
    MinLengthLogitsProcessor,
)
from transformers.generation_stopping_criteria import (
    MaxLengthCriteria,
    StoppingCriteriaList,
)

from group_bs import group_beam_search, expand_inputs_for_generation

In [2]:
SCRATCH = os.environ['SCRATCH']

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


#### Load Tokenizer

In [4]:
model_name_or_path = os.path.join(SCRATCH, 'huggingface/gpt2')
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

#### Test on One Prompt

In [5]:
FEW_SHOT_EXAMPLES = [
    {"Q": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
     "S": "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.",
     "A": "The answer is 6."},
    {"Q": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
     "S": "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.",
     "A": "The answer is 5."},
    {"Q": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
     "S": "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.",
     "A": "The answer is 39."},
    {"Q": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
     "S": "Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.",
     "A": "The answer is 8."},
    {"Q": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
     "S": "Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.",
     "A": "The answer is 9."},
    {"Q": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
     "S": "There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29.",
     "A": "The answer is 29."},
    {"Q": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
     "S": "Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls.",
     "A": "The answer is 33."},
    {"Q": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
     "S": "Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8.",
     "A": "The answer is 8."}
]

In [6]:
def build_prompt_with_few_shot_examples(question, n_shot=5):
    # creat context
    examples = ["Question: {}\nAnswer: {}".format(e["Q"], " ".join([e["S"], e["A"]])) for e in FEW_SHOT_EXAMPLES[:n_shot]]
    context = "\n\n".join(examples)
    
    return "\n\n".join([context, "Question: {}\nAnswer:".format(question)])

In [7]:
question = """Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. \
Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. \
How much more money does Betty need to buy the wallet?"""

prompt_text = build_prompt_with_few_shot_examples(question, n_shot=5)
print(prompt_text)

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
Answer: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Answer: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Answer: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Answer: Jason started with 20 lollipops. Then he had 12 

In [8]:
encoded_prompt = tokenizer(prompt_text, padding='longest', add_special_tokens=False, return_tensors="pt")
encoded_prompt = encoded_prompt.to(device)

input_ids = encoded_prompt['input_ids']
attention_mask = encoded_prompt['attention_mask']

In [9]:
# instantiate beam scorer
NUM_BEAM, NUM_GROUP = 6, 3
beam_scorer = BeamSearchScorer(
    batch_size=1,
    num_beams=NUM_BEAM,
    device=device,
    num_beam_groups=NUM_GROUP,
    num_beam_hyps_to_keep=NUM_BEAM,
)

# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        HammingDiversityLogitsProcessor(5.0, num_beams=NUM_BEAM, num_beam_groups=NUM_GROUP),
        MinLengthLogitsProcessor(5, eos_token_id=tokenizer.eos_token_id),
    ]
)

stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=100 + input_ids.shape[1])])

input_ids_expanded, model_kwargs = expand_inputs_for_generation(
    input_ids, expand_size=NUM_BEAM
)
with torch.no_grad():
    outputs = group_beam_search(
        tokenizer,
        input_ids_expanded,
        beam_scorer,
        logits_processor=logits_processor,
        stopping_criteria=stopping_criteria,
        parallel_generation=True,
        **model_kwargs,
    )

In [10]:
tokenizer.batch_decode(outputs[:, input_ids.shape[1]:], skip_special_tokens=True)

[' Betty needs $100 for the wallet. She has half of that, which is $50. Her parents give her $15, and her grandparents give her $30. So she has $50 + $15 + $30 = $95. She needs $100 - $95 = $5. The answer is $5.',
 ' Betty needs $100 for the wallet. She has half of that, which is $50. Her parents give her $15, and her grandparents give her $30. So she has $50 + $15 + $30 = $95. She needs $100 - $95 = $5 more. The answer is $5.',
 ' Betty needs $100 for the wallet. She has half of that, which is $50. Her parents give her $15, and her grandparents give her $30. So she has $50 + $15 + $30 = $95. She needs $100 - $95 = $5 more. The answer is $5 more.',
 ' Betty needs $100 for the wallet. She has half of that, which is $50. Her parents give her $15, and her grandparents give her $30. So she has $50 + $15 + $30 = $95. She needs $100 - $95 = $5 more. The answer is $5.\n',
 ' Betty needs $100 for the wallet. She has half of that, which is $50. Her parents give her $15, and her grandparents giv